# N Queens Problem with Forward Checking

In [1]:
from typing import List

def solve_nqueens_with_forward_checking(n: int) -> List[List[int]]:
    """
    Solves the N-Queens problem using backtracking with forward checking.

    Args:
        n: The size of the board and the number of queens.

    Returns:
        A list of all possible solutions, where each solution is a list
        representing the column position of the queen in each row.
    """
    solutions = []
    # `domains` stores the list of possible columns for the queen in each row.
    initial_domains = [list(range(n)) for _ in range(n)]

    def solve(board: List[int], row: int, domains: List[List[int]]):
        # Base case: If all queens are placed, we found a solution.
        if row == n:
            solutions.append(board[:])
            return

        # Try placing a queen in each available column in the current row's domain.
        for col in domains[row]:
            board[row] = col
            
            # Create a deep copy of domains to modify for the next state.
            new_domains = [d.copy() for d in domains]
            
            # --- Forward Checking Step ---
            # Prune the domains of all future rows based on the current queen placement.
            for future_row in range(row + 1, n):
                # 1. Remove the current column from future domains.
                if col in new_domains[future_row]:
                    new_domains[future_row].remove(col)
                
                # 2. Remove diagonal attacks.
                dist = future_row - row
                diag1 = col - dist
                diag2 = col + dist
                
                if diag1 in new_domains[future_row]:
                    new_domains[future_row].remove(diag1)
                if diag2 in new_domains[future_row]:
                    new_domains[future_row].remove(diag2)

            # Check if any future domain has become empty. If so, this path is invalid.
            if all(new_domains[r] for r in range(row + 1, n)):
                # If all future domains are still valid, recurse.
                solve(board, row + 1, new_domains)

    # Start the recursive solver.
    solve([-1] * n, 0, initial_domains)
    return solutions

# --- Example Usage ---
if __name__ == "__main__":
    n = 8
    all_solutions = solve_nqueens_with_forward_checking(n)
    
    print(f"N-Queens Problem (N={n})")
    print(f"Number of solutions found: {len(all_solutions)}")
    if all_solutions:
        print(f"One solution: {all_solutions[0]}")
        # To print the board for the first solution:
        for row in range(n):
            line = ['.'] * n
            line[all_solutions[0][row]] = 'Q'
            print(" ".join(line))

N-Queens Problem (N=8)
Number of solutions found: 92
One solution: [0, 4, 7, 5, 2, 6, 1, 3]
Q . . . . . . .
. . . . Q . . .
. . . . . . . Q
. . . . . Q . .
. . Q . . . . .
. . . . . . Q .
. Q . . . . . .
. . . Q . . . .


# Sudoku with Forward Checking

In [2]:
from typing import List, Tuple, Optional, Dict

Board = List[List[int]]
Domains = Dict[Tuple[int, int], List[int]]

def solve_sudoku_with_forward_checking(board: Board) -> bool:
    """
    Solves a Sudoku puzzle using backtracking with forward checking.

    Args:
        board: A 9x9 list of lists representing the puzzle, with 0 for empty cells.

    Returns:
        True if a solution is found (board is modified in-place), False otherwise.
    """
    
    def get_initial_domains(board: Board) -> Optional[Domains]:
        """Create the initial domains for all empty cells."""
        domains: Domains = {}
        for r in range(9):
            for c in range(9):
                if board[r][c] == 0:
                    # Start with all possible numbers
                    possibilities = list(range(1, 10))
                    # Remove numbers already in the same row, col, or box
                    for i in range(9):
                        # Row and Column check
                        if board[r][i] in possibilities: possibilities.remove(board[r][i])
                        if board[i][c] in possibilities: possibilities.remove(board[i][c])
                    
                    # 3x3 Box check
                    start_row, start_col = 3 * (r // 3), 3 * (c // 3)
                    for br in range(start_row, start_row + 3):
                        for bc in range(start_col, start_col + 3):
                            if board[br][bc] in possibilities:
                                possibilities.remove(board[br][bc])
                    
                    if not possibilities: return None # No solution possible from start
                    domains[(r, c)] = possibilities
        return domains

    def solve(board: Board, domains: Domains) -> bool:
        # Base case: If no empty cells are left, puzzle is solved.
        if not domains:
            return True

        # Heuristic: Pick the cell with the smallest domain to try next.
        r, c = min(domains, key=lambda k: len(domains[k]))

        # Try each possible number in the chosen cell's domain.
        for num in domains[(r, c)]:
            board[r][c] = num
            
            # Create a copy of domains to propagate constraints.
            new_domains = {k: v[:] for k, v in domains.items()}
            del new_domains[(r, c)] # Cell (r, c) is no longer empty

            # --- Forward Checking Step ---
            # Check if placing 'num' makes any peer's domain empty.
            empty_domain_found = False
            for peer_r, peer_c in get_peers(r, c):
                if (peer_r, peer_c) in new_domains and num in new_domains[(peer_r, peer_c)]:
                    new_domains[(peer_r, peer_c)].remove(num)
                    if not new_domains[(peer_r, peer_c)]:
                        empty_domain_found = True
                        break
            
            if empty_domain_found:
                continue # This 'num' is not valid, try the next one.
            
            # If successful, recurse with the updated board and domains.
            if solve(board, new_domains):
                return True
        
        # Backtrack if no number from the domain leads to a solution.
        board[r][c] = 0
        return False

    def get_peers(r: int, c: int) -> List[Tuple[int, int]]:
        """Returns all cells in the same row, column, and 3x3 box."""
        peers = set()
        for i in range(9):
            peers.add((r, i))
            peers.add((i, c))
        start_row, start_col = 3 * (r // 3), 3 * (c // 3)
        for br in range(start_row, start_row + 3):
            for bc in range(start_col, start_col + 3):
                peers.add((br, bc))
        peers.remove((r, c))
        return list(peers)

    initial_domains = get_initial_domains(board)
    if initial_domains is None:
        return False
    return solve(board, initial_domains)

# --- Example Usage ---
if __name__ == "__main__":
    sudoku_board = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    print("\nSudoku Puzzle (with Forward Checking)")
    print("Original board:")
    for row in sudoku_board: print(row)

    if solve_sudoku_with_forward_checking(sudoku_board):
        print("\nSolution found:")
        for row in sudoku_board:
            print(row)
    else:
        print("\nNo solution exists.")


Sudoku Puzzle (with Forward Checking)
Original board:
[5, 3, 0, 0, 7, 0, 0, 0, 0]
[6, 0, 0, 1, 9, 5, 0, 0, 0]
[0, 9, 8, 0, 0, 0, 0, 6, 0]
[8, 0, 0, 0, 6, 0, 0, 0, 3]
[4, 0, 0, 8, 0, 3, 0, 0, 1]
[7, 0, 0, 0, 2, 0, 0, 0, 6]
[0, 6, 0, 0, 0, 0, 2, 8, 0]
[0, 0, 0, 4, 1, 9, 0, 0, 5]
[0, 0, 0, 0, 8, 0, 0, 7, 9]

Solution found:
[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


# Map Coloring with Forward Checking

In [ ]:


from typing import Dict, List, Set, Optional

# --- Type Aliases for Readability ---
Color = str
Region = str
Assignment = Dict[Region, Color]
Domains = Dict[Region, Set[Color]]
Neighbors = Dict[Region, List[Region]]

def solve_map_coloring(
    regions: List[Region],
    neighbors: Neighbors,
    colors: List[Color]
) -> Optional[Assignment]:
    """
    Solves a map coloring problem using backtracking with forward checking.

    Args:
        regions: A list of all region names.
        neighbors: A dictionary mapping each region to its list of neighbors.
        colors: A list of available colors.

    Returns:
        A dictionary representing a valid color assignment, or None if no solution exists.
    """
    # Initialize domains: each region can initially be any color.
    initial_domains: Domains = {region: set(colors) for region in regions}

    def backtrack(assignment: Assignment, domains: Domains) -> Optional[Assignment]:
        """Recursive helper function to find a solution."""
        
        # Base case: If all regions are assigned a color, we have a solution.
        if len(assignment) == len(regions):
            return assignment

        # Heuristic: Select the unassigned region with the fewest remaining colors (MRV).
        unassigned_regions = [r for r in regions if r not in assignment]
        region = min(unassigned_regions, key=lambda r: len(domains[r]))

        # Try assigning each valid color from the region's current domain.
        for color in domains[region]:
            # 1. Assign the color
            assignment[region] = color
            
            # 2. --- Forward Checking Step ---
            # Create a deep copy of domains to propagate constraints.
            new_domains = {r: d.copy() for r, d in domains.items()}
            
            # Prune the chosen color from the domains of all neighbors.
            empty_domain_found = False
            for neighbor in neighbors[region]:
                if neighbor not in assignment and color in new_domains[neighbor]:
                    new_domains[neighbor].remove(color)
                    # If a neighbor's domain becomes empty, this assignment is invalid.
                    if not new_domains[neighbor]:
                        empty_domain_found = True
                        break
            
            # 3. Recurse if the forward check was successful (no empty domains created).
            if not empty_domain_found:
                result = backtrack(assignment.copy(), new_domains)
                if result is not None:
                    return result
        
        # 4. Backtrack: No valid color was found for this region.
        return None

    # Start the backtracking process with an empty assignment and initial domains.
    return backtrack({}, initial_domains)

# --- Example Usage: Australian Map Coloring ---
if __name__ == "__main__":
    # Define the regions and their neighbors
    australian_regions = ['WA', 'NT', 'SA', 'Q', 'NSW', 'V', 'T']
    australian_neighbors: Neighbors = {
        'WA': ['NT', 'SA'],
        'NT': ['WA', 'SA', 'Q'],
        'SA': ['WA', 'NT', 'Q', 'NSW', 'V'],
        'Q':  ['NT', 'SA', 'NSW'],
        'NSW':['Q', 'SA', 'V'],
        'V':  ['SA', 'NSW'],
        'T':  []
    }
    
    # Define the set of available colors
    available_colors = ['Red', 'Green', 'Blue']

    print("Solving Australian map coloring problem...")
    solution = solve_map_coloring(australian_regions, australian_neighbors, available_colors)

    if solution:
        print("\nSolution found! ✅")
        for region, color in sorted(solution.items()):
            print(f"- {region}: {color}")
    else:
        print("\nNo solution found. ❌")